In [3]:
import pandas as pd
import os
import joblib
from sklearn.ensemble import RandomForestClassifier

# === Пути ===
model_path = "models/random_forest.pkl"
os.makedirs("models", exist_ok=True)

# === Проверяем, существует ли модель ===
if os.path.exists(model_path):
    print(f"Загружаем модель из {model_path}")
    model = joblib.load(model_path)

else:
    print("Обучаем модель RandomForestClassifier...")

    # Загрузка данных
    events_train = pd.read_csv("data/events_train.csv", parse_dates=["timestamp"])
    user_item_features = pd.read_csv("data/user_item_features.csv")

    # Создание метки
    events_train["target"] = (events_train["event"] == "transaction").astype(int)
    train_df = events_train[["visitorid", "itemid", "target"]].drop_duplicates()

    # Объединение с фичами
    train_merged = train_df.merge(user_item_features, on=["visitorid", "itemid"], how="left").dropna()

    # Разделение
    X_train = train_merged.drop(columns=["visitorid", "itemid", "target", "last_interaction"])
    y_train = train_merged["target"]

    # Обучение
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Сохраняем модель
    joblib.dump(model, model_path)
    print(f"Модель сохранена в {model_path}")

Обучаем модель RandomForestClassifier...
Модель сохранена в models/random_forest.pkl


In [4]:
# Загрузка модели
model = joblib.load("models/random_forest.pkl")

# Загрузка user-item признаков и тестовой выборки
user_item_features = pd.read_csv("data/user_item_features.csv")
events_test = pd.read_csv("data/events_test.csv")

# Формируем уникальные (visitorid, itemid)
test_pairs = events_test[["visitorid", "itemid"]].drop_duplicates()

In [5]:
# Объединяем с user_item_features
test_data = test_pairs.merge(user_item_features, on=["visitorid", "itemid"], how="left")

# Удаляем строки без признаков (если такие есть)
test_data = test_data.dropna()

In [6]:
# Удаляем ненужные колонки перед предсказанием
X_test = test_data.drop(columns=["visitorid", "itemid", "last_interaction"])

# Предсказываем вероятность покупки
test_data["score"] = model.predict_proba(X_test)[:, 1]

In [7]:
top3 = (
    test_data.sort_values(["visitorid", "score"], ascending=[True, False])
    .groupby("visitorid")
    .head(3)
    .reset_index(drop=True)
)

# Сохраняем рекомендации
top3[["visitorid", "itemid", "score"]].to_csv("models/recommendations_top3.csv", index=False)
print("Рекомендации сохранены в recommendations_top3.csv")

Рекомендации сохранены в recommendations_top3.csv
